In [1]:
import pandas as pd
from tqdm import tqdm
import os
import time
import numpy as np

import sys
sys.path.append('../mlrose')
from mlrose_hiive.generators.max_k_color_generator import MaxKColorGenerator
from mlrose_hiive import GARunner

In [2]:
ALGORITHM = 'ga'
PROBLEM_SIZE = 300
CONNECTIONS_PER_NODE = 3
MAX_COLORS = 4
MAX_ITERATIONS = 100
MAX_ATTEMPTS = 100
NUM_RUNS = 3

MUTATION_RATE_LIST = [0.1, 0.3, 0.5, 0.7]

POPULATION_SIZES_LIST = [10, 100, 500, 1000]

In [3]:
# SEED = 6
SEED = int(time.time())
problem = MaxKColorGenerator().generate(seed=SEED, number_of_nodes=PROBLEM_SIZE, 
                                        max_connections_per_node=CONNECTIONS_PER_NODE, 
                                        maximize=True, max_colors=MAX_COLORS)

runner = GARunner(
    problem=problem,
    experiment_name='GA',
    output_directory='.',
    seed=SEED,
    iteration_list=[MAX_ITERATIONS],
    max_attempts=MAX_ATTEMPTS,
    population_sizes=[10],
    mutation_rates=[0.1],
)
x, df_run_curves = runner.run()

In [4]:
print(df_run_curves['Fitness'].max())
df_run_curves.head()

498.0


,Iteration,Time,Fitness,FEvals,Population Size,Mutation Rate,max_iters
0,0,0.001956,416.0,10.0,10,0.1,100
1,1,0.007359,443.0,22.0,10,0.1,100
2,2,0.009402,459.0,34.0,10,0.1,100
3,3,0.011330,459.0,45.0,10,0.1,100
4,4,0.013225,459.0,56.0,10,0.1,100


In [5]:
df_path = f'metrics/{ALGORITHM}.csv'
os.makedirs(os.path.dirname(df_path), exist_ok=True)
if not os.path.exists(df_path):
    all_df = pd.DataFrame(columns=[
        # runner columns
        'Iteration', 'Time', 'Fitness', 'FEvals', 'Population Size', 'Mutation Rate', 'max_iters',
        # problem config
        'problem_size', 'connections_per_node', 'max_attempts', 'num_edges',
        # run config
        'run_number', 'group_timestamp', 'total_runtime',
    ])
    all_df.to_csv(df_path, index=False)
all_df = pd.read_csv(df_path)

In [6]:
for mutation in MUTATION_RATE_LIST:
    print(f'Mutation: {mutation}')
    for population in POPULATION_SIZES_LIST:
        print(f"Population: {population}")

        if (
            (all_df['problem_size'] == PROBLEM_SIZE) &
            (all_df['connections_per_node'] == CONNECTIONS_PER_NODE) &
            (all_df['Population Size'] == population) &
            (all_df['Mutation Rate'] == mutation) &
            (all_df['max_attempts'] == MAX_ATTEMPTS) &
            (all_df['max_iters'] == MAX_ITERATIONS)
        ).any():
            print('Already run')
            continue

        group_timestamp = int(time.time())
        time.sleep(1)
        max_fitnesses = []
        for run_i in tqdm(range(NUM_RUNS)):
            t_start = time.perf_counter()
            problem = MaxKColorGenerator().generate(seed=run_i, number_of_nodes=PROBLEM_SIZE, 
                                                    max_connections_per_node=CONNECTIONS_PER_NODE, 
                                                    maximize=True, max_colors=MAX_COLORS)
            runner = GARunner(
                problem=problem,
                experiment_name='GA',
                output_directory='.',
                seed=run_i,
                iteration_list=[MAX_ITERATIONS],
                max_attempts=MAX_ATTEMPTS,
                population_sizes=[population],
                mutation_rates=[mutation],
            )
            _, df_run_curves = runner.run()

            df_run_curves['problem_size'] = PROBLEM_SIZE
            df_run_curves['connections_per_node'] = CONNECTIONS_PER_NODE
            df_run_curves['max_attempts'] = MAX_ATTEMPTS

            df_run_curves['Population Size'] = population
            df_run_curves['Mutation Rate'] = mutation
            df_run_curves['max_iters'] = MAX_ITERATIONS

            df_run_curves['run_number'] = run_i
            df_run_curves['group_timestamp'] = group_timestamp
            df_run_curves['num_edges'] = problem.length

            # print(f"Max Fitness: {df_run_curves['Fitness'].max()}")
            # print(f"Max Iteration: {df_run_curves['Iteration'].max()}")

            max_fitnesses.append(df_run_curves['Fitness'].max())

            t_end = time.perf_counter()
            df_run_curves['total_runtime'] = t_end - t_start

            all_df = pd.concat([all_df, df_run_curves])

        print(f'Max Fitness: {np.mean(max_fitnesses)}')

        all_df.reset_index(inplace=True, drop=True)
        all_df.to_csv(df_path, index=False)

Mutation: 0.1
Population: 10


100%|██████████| 3/3 [01:46<00:00, 35.46s/it]


Max Fitness: 507.3333333333333
Population: 100


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
all_df.columns

In [ ]:
agg_df = pd.DataFrame()
for group_timestamp in all_df['group_timestamp'].unique():
    df = all_df[all_df['group_timestamp'] == group_timestamp]
    max_fit = df['Fitness'].max()
    total_iters = df['Iteration'].max()
    max_fevals = df['FEvals'].max()
    df.drop(columns=['Fitness', 'Iteration', 'FEvals', 'Time'], inplace=True)
    assert(len(df) > NUM_RUNS)
    df.drop_duplicates(inplace=True)
    assert(len(df) == NUM_RUNS)
    df['max_fitness'] = max_fit
    df['total_iterations'] = total_iters
    df['max_fevals'] = max_fevals
    agg_df = pd.concat([agg_df, df])
agg_df.reset_index(inplace=True, drop=True)

In [ ]:
df_path = f'metrics/{ALGORITHM}_agg.csv'
agg_df.to_csv(df_path, index=False)